# Pytorch Dense Net tabular data train with batches

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

class SigModel(torch.nn.Module):
    def __init__(self, input_channels):
        super(SigModel, self).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module('lin1', torch.nn.Linear(input_channels, 64))
        self.layers.add_module('relu1', torch.nn.ReLU())
        self.layers.add_module('do1', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin2', torch.nn.Linear(64, 128))
        self.layers.add_module('relu2', torch.nn.ReLU())
        self.layers.add_module('do2', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin3', torch.nn.Linear(128, 96))
        self.layers.add_module('relu3', torch.nn.ReLU())
        self.layers.add_module('do3', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin4', torch.nn.Linear(96, 32))
        self.layers.add_module('relu4', torch.nn.ReLU())
        self.layers.add_module('do4', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin5', torch.nn.Linear(32, 1))
        self.layers.add_module('sigmoid', torch.nn.Sigmoid())
    
    def forward(self, input):
        return self.layers(input)

# Define your custom dataset for loading tabular data
class TabularDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    
    def __len__(self):
        return len(self.data)

# Prepare your tabular data
features = [...] # Select your features
data = df[features] # Your tabular data, shape: (num_samples, num_features)
labels = df['target'] # Your corresponding labels, shape: (num_samples,)
input_channels = data.shape[1]
output_size = len(set(labels))

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create data loaders
batch_size = len(train_data) // 10
train_dataset = TabularDataset(train_data.values, train_labels.values)
val_dataset = TabularDataset(val_data.values, val_labels.values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Create an instance of your fully convolutional network
net = SigModel(input_channels)

# Define the loss function and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    net.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(torch.float32).to(device)
        labels = labels.to(torch.float32).to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs).squeeze()

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Validate the model
    net.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(torch.float32).to(device)
            labels = labels.to(torch.float32).to(device)

            outputs = net(inputs).squeeze()
            val_loss += criterion(outputs, labels).item()

            total += labels.size(0)
            correct += (outputs == labels).sum().item()

    # Print loss and accuracy for this epoch
    print(f"Epoch {epoch+1}: Training Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {100*correct/total:.2f}%")

# Pytorch Dense Net tabular data train without batches

### Create dataset

In [3]:
from torch import nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from sklearn.preprocessing import StandardScaler

test_size=0.2

features = [...] # Select your features
x_data = df[features] # Your tabular data, shape: (num_samples, num_features)
y_data = df['target'] # Your corresponding labels, shape: (num_samples,)
input_channels = x_data.shape[1]
output_size = len(set(y_data))

x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=test_size, shuffle=True)

scaler = StandardScaler()
x_train[x_train.columns] = scaler.fit_transform(x_train)
x_valid[x_valid.columns] = scaler.transform(x_valid)

x_train = torch.tensor(x_train.values, dtype=torch.float32)
x_valid = torch.tensor(x_valid.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_valid = torch.tensor(y_valid.values, dtype=torch.float32)

display(type(x_train), type(y_train))

# find available device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Build model

In [ ]:
class SigModel(torch.nn.Module):
    def __init__(self, input_channels):
        super(SigModel, self, ).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module('lin1', torch.nn.Linear(input_channels, 64))
        self.layers.add_module('relu1', torch.nn.ReLU())
        self.layers.add_module('do1', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin2', torch.nn.Linear(64, 128))
        self.layers.add_module('relu2', torch.nn.ReLU())
        self.layers.add_module('do2', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin3', torch.nn.Linear(128, 96))
        self.layers.add_module('relu3', torch.nn.ReLU())
        self.layers.add_module('do3', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin4', torch.nn.Linear(96, 32))
        self.layers.add_module('relu4', torch.nn.ReLU())
        self.layers.add_module('do4', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin5', torch.nn.Linear(32, 1))
        self.layers.add_module('sigmoid', torch.nn.Sigmoid())
    
    def forward(self, input):
        return self.layers(input)

### Train model with Optimizer and Scheduler

In [ ]:
# train function
def train_epoch(model, train_loader, criterion, optimizer, scheduler):
    # put the model in train mode
    model.train()
    
    x_train, x_valid, y_train, y_valid = train_loader

    # get output of the model
    train_preds = model(x_train).squeeze()
    # calculate train loss
    train_loss = criterion(train_preds, y_train)
    train_acc = (train_preds.round() == y_train).float().mean()
    
    # set gradient to zero to prevent it accumulation
    optimizer.zero_grad() # ~ model.zero_grad()
    # calculate gradient
    train_loss.backward() 
    # update weights
    optimizer.step()
    
    # put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        val_preds = model(x_valid).squeeze()
        val_loss = criterion(val_preds, y_valid)
        val_acc = (val_preds.round() == y_valid).float().mean()
    
    # update weights according to gradient value
    scheduler.step(val_loss)
    
    return train_loss, train_acc, val_loss, val_acc

# Initialize model
model = SigModel(input_channels).to(device)

# Number of epochs
epochs = 10000

# Send data to the device
x_train, x_valid = x_train.to(device), x_valid.to(device)
y_train, y_valid = y_train.to(device), y_valid.to(device)
train_loader = x_train, x_valid, y_train, y_valid

# Empty loss lists to track values
epoch_count, train_loss_values, valid_loss_values = [], [], []

criterion = nn.BCELoss()
learning_rate = 1e-6
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2000, threshold=1e-2)

# Loop through the data
for epoch in range(epochs):
    train_loss, train_acc, val_loss, val_acc = train_epoch(model, train_loader, criterion, optimizer, scheduler)

    # Print progress a total of 20 times
    if epoch % int(epochs / 20) == 0:
        print(f'Epoch: {epoch:4.0f} | Train Loss: {train_loss:.5f}, Train Acc: {train_acc:.5f}\
            Validation Loss: {val_loss:.5f}, Val Acc: {val_acc:.5f}\
                LR: {optimizer.state_dict()["param_groups"][0]["lr"]}')

        epoch_count.append(epoch)
        train_loss_values.append(train_loss.cpu().detach().numpy())
        valid_loss_values.append(val_loss.cpu().detach().numpy())


### Plot NN train results

In [ ]:
import matplotlib.pyplot as plt

plt.plot(epoch_count, train_loss_values, label='Training Loss')
plt.plot(epoch_count, valid_loss_values, label='Validation Loss')
plt.title('Training & Validation Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()